# Classifier for Inception Score

by Búgvi Benjamin Magnussen and Nikolaj Bläser

In [ ]:
import numpy as np
import tensorflow as tf
from matplotlib import pyplot as plt

# Data preprocessing

In [ ]:
mnist = tf.keras.datasets.mnist.load_data()
(X_train, Y_train), (X_test_validation, Y_test_validation) = mnist
Y_train = tf.keras.utils.to_categorical(Y_train)
Y_test_validation = tf.keras.utils.to_categorical(Y_test_validation)

# Dividing the second dataset into test and validation sets
X_validation = X_test_validation[len(X_test_validation)//2:]
X_test = X_test_validation[:len(X_test_validation)//2]

Y_validation = Y_test_validation[len(Y_test_validation)//2:]
Y_test = Y_test_validation[:len(Y_test_validation)//2]


In [ ]:
def normalize(images): 
  return (images - 127.5) / 127.5

In [ ]:
X_train = normalize(X_train)
X_validation = normalize(X_validation)
X_test = normalize(X_test)

In [ ]:
image = X_train[0]
IMAGE_WIDTH = image.shape[0]
IMAGE_HEIGHT = image.shape[1]

In [ ]:
BATCH_SIZE = 256
train = tf.data.Dataset.from_tensor_slices(tuple((X_train, Y_train))).shuffle(len(X_train)).batch(BATCH_SIZE)

# Activation Functions

In [ ]:
# The code is from tensorflow.contrib.layers.maxout
import tensorflow as tf
def maxout(inputs, num_units, axis=-1):
  '''
  inputs: Tensor input
  num_units: The num of unit keeped after amxout
  axis: The dimension max op performed
  scope: Optional scope for variable_scope
     Note: This is a slightly modified version. Replaced some unused API functions
  '''
  shape = inputs.get_shape().as_list()
  num_channels = shape[axis]
  if num_channels % num_units:
    raise ValueError('number of features({}) is not '
                      'a multiple of num_units({})'.format(
                          num_channels, num_units))
  shape[axis] = -1
  shape += [num_channels // num_units]

  # Dealing with batches with arbitrary sizes
  for i in range(len(shape)): # This is used to handle the case where None is included in the shape
    if shape[i] is None:
      shape[i] = tf.shape(inputs)[i]
  outputs = tf.reduce_max( tf.reshape(inputs, shape), -1)
  return outputs

# Layer Functions

In [ ]:
def denseRelu(inputs, weights, bias, leaky_relu_alpha = 0.2):
  return tf.nn.leaky_relu(tf.nn.bias_add(tf.matmul(inputs, weights), bias), alpha=leaky_relu_alpha)

def denseMaxout(inputs, weights, bias, num_of_units=2, dropout_rate=0.5):
  z = tf.nn.bias_add(tf.matmul(inputs, weights), bias)
  z_dropout = tf.nn.dropout(z, rate=dropout_rate)
  return maxout(z_dropout, num_of_units)

# Parameter Initialization

In [ ]:
initializer = tf.initializers.glorot_normal()
bias_initializer = tf.initializers.zeros()

def get_biases(n_units, name):
  return tf.Variable(bias_initializer(n_units, dtype=tf.float64), name = name, trainable = True, dtype=tf.float64)


def get_weights(shape, name):
  return tf.Variable(initializer(shape, dtype=tf.float64), name = name, trainable = True, dtype=tf.float64)

In [ ]:
biases = [
  get_biases(1200, 'bias0'),
  get_biases(1200, 'bias1'),
  get_biases(10, 'bias2')
]

weights = [
  get_weights([image.shape[0] * image.shape[1], 1200], 'weights0'),
  get_weights([240, 1200], 'weights1'),
  get_weights([240, 10], 'weights2'),
]

parameters = weights + biases

# Model

In [ ]:
dropout_rate = 0.5

@tf.function
def classifier(x):
  x = tf.cast(x, dtype=tf.float64)
  x = tf.reshape(x, shape=[x.shape[0], x.shape[1] * x.shape[2]])
  d1 = denseMaxout(x, weights[0], biases[0], num_of_units=240, dropout_rate=dropout_rate)
  d2 = denseMaxout(d1, weights[1], biases[1], num_of_units=240, dropout_rate=dropout_rate)
  return tf.nn.softmax(tf.nn.bias_add(tf.matmul(d2, weights[2]), biases[2]))

# Measurement Functions

In [ ]:
def calculate_F1(input, true_labels, message):
  predicted = tf.convert_to_tensor([tf.one_hot(tf.argmax(t), depth = 10) for t in classifier(input)])
  predicted_T = predicted.numpy().T
  true_labels_T = true_labels.T
  total_positive = np.sum(predicted_T, axis=1)
  true_positive = np.sum([[predicted_T[i][j] == true_labels_T[i][j] == 1 for j in range(5000)] for i in range(10)], axis = 1)
  false_negative = np.sum([[abs(predicted_T[i][j]-1) == true_labels_T[i][j] == 1 for j in range(5000)] for i in range(10)], axis = 1)

  precision = np.average(true_positive / total_positive)
  recall = np.average(true_positive / (true_positive + false_negative))
  F_1 = 2 * ((precision * recall) / (precision + recall))
  print(message)
  print("Precision:", precision)
  print("Recall:", recall)
  print("F1:", F_1)

def accuracy(input, true_labels, message):
  predicted = tf.convert_to_tensor([tf.one_hot(tf.argmax(t), depth = 10) for t in classifier(input)])
  equal = [ (tf.argmax(predicted[i]) == tf.argmax(true_labels[i])).numpy() for i in range(len(predicted))]
  print(message, sum(equal)/len(equal))

# Hyperparameters

In [ ]:
cross_entropy = tf.keras.losses.CategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam(1e-4)
EPOCHS = 300

# Training

In [ ]:
def train_step(images, true_labels):
  with tf.GradientTape() as disc_tape:
    output = classifier(images)
    loss = cross_entropy(true_labels, output)
  
  gradients = disc_tape.gradient(loss, parameters)
  optimizer.apply_gradients(zip(gradients, parameters))
  return loss

In [ ]:
from IPython.display import clear_output

for e in range(EPOCHS):
  if(e + 1 % 10  == 0):
    calculate_F1(X_validation, Y_validation, "Validation set Measurements: ")
  loss_accumulator = 0
  for i, batch in enumerate(train):
    loss_accumulator += train_step(batch[0], batch[1])
  clear_output(wait=True)
  print("Epochs: " + str(e+1) + "\\" + str(EPOCHS))
  print("Loss: ", loss_accumulator.numpy()/len(train))
  loss_accumulator = 0

# Measurement for Test Set

In [ ]:
accuracy(X_test, Y_test, "Accuracy:")

In [ ]:
calculate_F1(X_test, Y_test, "Test set Measurements:")

# Saving the model

In [ ]:
np_weights = [w.numpy() for w in weights]
np_biases = [w.numpy() for w in biases]
np.save('./Classifier_params/weights.npy', np_weights)
np.save('./Classifier_params/biases.npy', np_biases)